In [1]:
!pip install pandas numpy tensorflow tensorflowjs

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [3]:
class TargetEncoding(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs) # Boilerplate
        
    def call(self, inputs):
#         return self.norm(self.table.lookup(inputs))
        return self.table.lookup(inputs)
    
    def compute_output_shape(self, input_shape):
        return input_shape
    
#     def compute_output_signature()
    def adapt(self, feature, target, 
              mean, var, default_value=-1):
        self.keys = tf.constant(feature)
        self.target = tf.constant(target)
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(self.keys,self.target),
            default_value=default_value)
#         self.norm = layers.Normalization(axis=None, 
#                                          mean=mean,
#                                          variance=var)

In [9]:
data = pd.DataFrame()
data['Target'] = np.random.normal(0,1,(100,))
data['Feature'] = ["Class A"] * 30 + ["Class B"] * 20 \
    + ["Class C"] * 50

In [19]:
gr = data.groupby('Feature')['Target'].mean()
targ_enc = TargetEncoding()
targ_enc.adapt(
    feature=gr.index,
    target=gr.values,
    mean=data.Target.mean(),
    var=data.Target.var(),
    default_value=data.Target.median(),
)

In [25]:
inputs = {
    'feature': keras.Input(shape=(1,), dtype=tf.string, name='feature')
}
enc = targ_enc(inputs['feature'])
model = keras.Model(inputs=inputs, outputs=enc)
model(data.Feature.values)[:5]

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[0.31229785],
       [0.31229785],
       [0.31229785],
       [0.31229785],
       [0.31229785]])>

In [28]:
tf.saved_model.save(model, 'models/tf_model/')

INFO:tensorflow:Assets written to: models/tf_model/assets


In [30]:
#     --signature_name=serving_default \
#     --control_flow_v2=True \
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_format=tfjs_graph_model \
    models/tf_model/ \
    models/tfjs_model/

2022-05-18 22:48:24.168276: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 22:48:24.198042: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 22:48:24.198232: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 22:48:24.198575: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags